In [86]:
from keras.models import Sequential
from keras.layers import *
from IPython.display import clear_output#to clear output
import time

## Steps

1. Loading Data
2. Import keras models
3. Add different layers using model.add<br>
Densely connected layer is one in which each neuron in every layer is connected to every other neuron in next layer
4. Compile Model
5. Fit model
6. Evaluate Model
7. Save model
8. Load Model
9. Predict using model

## Loading data 

Neural Network works best when data in entire datasent in scaled on some scale
Pandas act as though we are working in a <b>virtual spreadsheet</b>.We can do many of the same functions that we can do on a spreadsheet like sorting data, pulling out certain columns of data and or applying calculations

In [87]:

import pandas as pd
training_data_df=pd.read_csv("sales_data_training.csv")#training data data frame
test_data_df=pd.read_csv("sales_data_test.csv")#testing data data frame
print(training_data_df)
time.sleep(5)
clear_output()

## Scaling data
<b>minmax</b> object of sklearn library will help to scale data

In [88]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))#setting the range of scaling
scaled_training = scaler.fit_transform(training_data_df)#fit_transform returns plain numpy array and not pandas dataframe
scaled_testing = scaler.transform(test_data_df)#use the parameters computed above for scaling data
#finding how much each column is scaled in training
print(scaler.scale_)
#finding how much total training data is scaled
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

#reconverting numpy array scaled_training into pandas dataframe so that it can be written to file easily
scaled_training_df=pd.DataFrame(scaled_training,columns=training_data_df.columns.values)
scaled_testing_df=pd.DataFrame(scaled_testing,columns=training_data_df.columns.values)

#writing scaled training and testing data to .csv file
scaled_training_df.to_csv("sales_data_training_scaled.csv",index=False)
scaled_testing_df.to_csv("sales_data_testing_scaled.csv",index=False)


[3.33333333e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 3.69678934e-06 5.00000000e-02]
Note: total_earnings values were scaled by multiplying by 0.0000036968 and adding -0.115913


## Dividing training data into input and output

In [89]:
X=scaled_training_df.drop(columns='total_earnings',axis=1).values
Y=scaled_training_df[['total_earnings']].values

## Importing keras, creating model, compiling model 

In [90]:
model=Sequential()
model.add(Dense(32,input_dim=9,activation='relu'))
model.add(Dense(128))
model.add(Dense(50))
model.add(Dense(1,activation='linear'))#very imp to choose correct activation function. Act. fun is not sigmoid because this prob is not of logistic regression
model.compile(optimizer='adam',loss='mse')#adam is one of the gradient descent mechanism
#MSE (Y_actual(i)^2-Y_predicted(i)^2)/n

## Training Model

In [91]:
model.fit(
    X,
    Y, 
    epochs=50,  
    shuffle=True,
    verbose=0)
#X: actual data for prediction
#Y: actual output values
#epochs: a single training pass across the entire training data is called epoch
#verbose: print more detaled model about what is going on

## Evaluating the model
Model is evaluated on <b>test data</b>

### Dividing test data into input and output

In [92]:
scaled_testing_df=pd.read_csv("sales_data_testing_scaled.csv")
X_test=scaled_testing_df.drop(columns='total_earnings',axis=1).values
Y_test=scaled_testing_df[['total_earnings']].values

### Evaluating model on training and testing

In [93]:
test_error_rate=model.evaluate(X_test,Y_test,verbose=0)#the smaller the error rate, the better it is
print("MSE is {}".format(test_error_rate))

MSE is 0.00038611555704846977


## Making the predictions

In [94]:
X_Predict = pd.read_csv('proposed_new_product.csv').values
prediction = model.predict(X_Predict)
#We only want first element of the first prediction
Y_Predict = prediction[0][0]

### Scaling back data is very imp. The scaling operations must be performed in reverse order### Multiplication becomes division and addition becomes subtraction

In [95]:
#Scaling back predicted values from [0, 1] to original scale
Y_Predict_Scaled=Y_Predict / 0.0000036968+ 0.115913
print('Predicted total earning of proposed product {}'.format(Y_Predict_Scaled))

Predicted total earning of proposed product 224965.80995600738


# Saving trained model
large neural networks takes hours/days in training. We cannot retrain them for each time for execution. We have to save the trained model and load the trained model if and when required

In [96]:
model.save('trained_model.h5')#save the complete model
#h5 is binary file format used to store python array objects

# Loading saved trained model

In [97]:
from keras.models import load_model
loaded_model = load_model('trained_model.h5')

### Ensuring that model is working fine after loading

In [98]:
#testing whether loaded model is working correctly or not
X_Predict = pd.read_csv('proposed_new_product.csv').values
prediction = loaded_model.predict(X_Predict)
#We only want first element of the first prediction
Y_Predict = prediction[0][0]
#Scaling back predicted values from [0, 1] to original scale
Y_Predict_Scaled=Y_Predict / 0.0000036968+ 0.115913
print('Predicted total earning of proposed product {}'.format(Y_Predict_Scaled))

Predicted total earning of proposed product 224965.80995600738
